# Import custom images and annotations

In [16]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import boto3
import io
import pandas as pd
import json
#!pip3 install s3fs
import s3fs

my_bucket = 'autoarki-ground-truth-labeling'
client = boto3.client('s3')
response = client.list_objects_v2(Bucket=my_bucket)


ImportError: cannot import name 'ParamSpec'

## Gather list of annotated and non-annotated files

In [8]:
# list of all annotated files
bucket = 'autoarki-ground-truth-labeling'
dest = "bounding_box/ground_truth_annots/yolo-bbox-train"
file_key = "annot.csv"
annot_file = 's3://{}/{}/{}'.format(bucket, dest, file_key)
df_ann = pd.read_csv(annot_file)

annotated_img_list = list(df_ann.img_file.unique())

ImportError: Missing optional dependency 's3fs'. The s3fs package is required to handle s3 files. Use pip or conda to install s3fs.

In [ ]:
# list of all files that were reviewed in annotation job
import s3fs

reviewed_img_list = []

mani_path = "s3://autoarki-ground-truth-labeling/bounding_box/ground_truth_annots/yolo-bbox-train/manifests/output.manifest"
job_name = 'yolo-bbox-train'

fs = s3fs.S3FileSystem(anon=False)
with fs.open(mani_path, 'rb') as fin:    
    annot_list = []

    for line in fin.readlines():
        record = json.loads(line)
        if job_name in record.keys():  # is it necessary?
            image_file_path = record["source-ref"]
            image_file_name = image_file_path.split("/")[-1]
            reviewed_img_list.append(image_file_name)
            

In [ ]:
print("Number of reviewed files: ", len(reviewed_img_list))
print("Number of annotated files: ", len(annotated_img_list))

## Class Balance

In [ ]:
# remove all annotated files from list
non_annot_review_list = [x for x in reviewed_img_list if x not in annotated_img_list]
print("Number of non-annotated but reviewed files: ", len(non_annot_review_list))

In [ ]:
# select 125% of the annotated files 

import random
seed = 123
random.Random(seed).shuffle(non_annot_review_list)

num_non_annotated = round(len(annotated_img_list)*1.25)

non_ann_to_use_list = non_annot_review_list[:num_non_annotated]
print("Number of non-annotated but reviewed files to be used in train + test: ", num_non_annotated)

## Split into Train and Test

Split annotated and non-annotated data separately into train and test then append

In [5]:
def split_train_test(full_list, train_pct = 0.8):
    '''Return list of files for train and list of files for test''' 
    split_1 = round(int(train_pct * len(full_list)),0)

    train_filenames = full_list[ :split_1]
    test_filenames = full_list[split_1: ]
    
    return train_filenames, test_filenames


In [6]:
train_files, test_files = split_train_test(annotated_img_list) # annotated list
non_ann_train, non_ann_test = split_train_test(non_ann_to_use_list)

train_files.extend(non_ann_train)
test_files.extend(non_ann_test)
print("Number of Training Files", len(train_files))
print("Number of Test Files", len(test_files))

NameError: name 'annotated_img_list' is not defined

# Create a subdirectory with copies of the files
Note: Delete originals once proof of concept is complete.


In [2]:
train_files[:3]

NameError: name 'train_files' is not defined

In [ ]:
train_dir = "s3://autoarki-ground-truth-labeling/training_data/"
test_dir = "s3://autoarki-ground-truth-labeling/test_data/"

image_file_location = "s3://autoarki-ground-truth-labeling/bounding_box/images/"
ann_text_file_location = 's3://autoarki-ground-truth-labeling/bounding_box/ground_truth_annots/yolo-bbox-train/yolo_friendly_format/'

In [ ]:
def copy_files_for_YOLO(list_file_names, new_dir):
    for i in list_file_names:
        '''Copy over txt and jpg files'''

        s3_resource.Object(my_bucket, new_dir).copy_from(CopySource=old_dir)

In [95]:
# Training subdirectory
len_filenames = len("_filenames")
s3_resource = boto3.resource('s3')

def move_to_new_directories(list_files, file_subdir = "train"):
    for file_name in list_files:
        new_dir = "s3://label-test3/" + file_subdir + '/raw_images/' + file_name
        old_dir = "s3://label-test3/" + file_name
        print(new_dir)
        #s3_resource.Object(bucket, new_dir).copy_from(CopySource=old_dir)

move_to_new_directories(train_filenames, "train")
#move_to_new_directories(val_filenames, "val")
# Delete the former object A
#client.Object(bucket, “path/to/your/object_A.txt”).delete()

s3://label-test3/train/raw_images/0029-0147.png
s3://label-test3/train/raw_images/0157-1224.png
s3://label-test3/train/raw_images/0018-0053.png
s3://label-test3/train/raw_images/0029-0128.png
s3://label-test3/train/raw_images/0010-0044.png
s3://label-test3/train/raw_images/0022-0045.png
s3://label-test3/train/raw_images/0010-0053.png
s3://label-test3/train/raw_images/0011-0043.png
s3://label-test3/train/raw_images/0029-0144.png
s3://label-test3/train/raw_images/0006-0053.png
s3://label-test3/train/raw_images/0060-0074.png
s3://label-test3/train/raw_images/0018-0045.png
s3://label-test3/train/raw_images/0025-0044.png
s3://label-test3/train/raw_images/0193-0025.png
s3://label-test3/train/raw_images/0010-0045.png
s3://label-test3/train/raw_images/0025-0043.png
s3://label-test3/train/raw_images/0141-0009.png
s3://label-test3/train/raw_images/0193-0028.png
s3://label-test3/train/raw_images/0011-0053.png
s3://label-test3/train/raw_images/0141-0014.png
s3://label-test3/train/raw_images/0025-0

# Learning

use smallest YOLO model to start yolov5s.pt

In [ ]:
#!pip install torch
#!pip install torchvision 
#!conda install pytorch torchvision -c pytorch


In [86]:
# change directory
import os
os.chdir(r"/root/AutoArki/yolov5/yolov5")
! python train.py\
    --data AutoArki/kineret - /root/AutoArki/kineret - AWS experiments/custom_dataset.yaml \
    --epochs 30 \
    --project custom_yolov5 \
    --bbox_interval 1 \
    --save_period 1 \
    --weights yolov5s.pt

Traceback (most recent call last):
  File "train.py", line 26, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


In [ ]:
# Train
train_dataset = KangarooDataset()
train_dataset.load_dataset(dataset_dir='./kangaroo-transfer-learning/kangaroo', is_train=True)
train_dataset.prepare()

# Validation
validation_dataset = KangarooDataset()
validation_dataset.load_dataset(dataset_dir='./kangaroo-transfer-learning/kangaroo', is_train=False)
validation_dataset.prepare()

In [ ]:
class KangarooConfig(mrcnn.config.Config):
    NAME = "kangaroo_cfg"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 2
    
    LEARNING_RATE = 0.001

    STEPS_PER_EPOCH = 131